In [1]:
import sys
import os
# Add the root directory of the nesymres module to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))


### Simple example for performing symbolic regression for a set of points

In [2]:
from nesymres.architectures.model import Model
from nesymres.utils import load_metadata_hdf5
from nesymres.dclasses import FitParams, NNEquation, BFGSParams
from pathlib import Path
from functools import partial
import torch
from sympy import lambdify
import json

In [3]:
## Load equation configuration and architecture configuration
import omegaconf
with open('100M/eq_setting.json', 'r') as json_file:
  eq_setting = json.load(json_file)

cfg = omegaconf.OmegaConf.load("100M/config.yaml")

In [4]:
## Set up BFGS load rom the hydra config yaml
bfgs = BFGSParams(
        activated= cfg.inference.bfgs.activated,
        n_restarts=cfg.inference.bfgs.n_restarts,
        add_coefficients_if_not_existing=cfg.inference.bfgs.add_coefficients_if_not_existing,
        normalization_o=cfg.inference.bfgs.normalization_o,
        idx_remove=cfg.inference.bfgs.idx_remove,
        normalization_type=cfg.inference.bfgs.normalization_type,
        stop_time=cfg.inference.bfgs.stop_time,
    )


In [5]:
params_fit = FitParams(word2id=eq_setting["word2id"], 
                            id2word={int(k): v for k,v in eq_setting["id2word"].items()}, 
                            una_ops=eq_setting["una_ops"], 
                            bin_ops=eq_setting["bin_ops"], 
                            total_variables=list(eq_setting["total_variables"]),  
                            total_coefficients=list(eq_setting["total_coefficients"]),
                            rewrite_functions=list(eq_setting["rewrite_functions"]),
                            bfgs=bfgs,
                            beam_size=cfg.inference.beam_size #This parameter is a tradeoff between accuracy and fitting time
                            )

In [6]:
weights_path = "../weights/100M.ckpt"

In [7]:
## Load architecture, set into eval mode, and pass the config parameters
model = Model.load_from_checkpoint(weights_path, cfg=cfg.architecture)
model.eval()
if torch.cuda.is_available(): 
  model.cuda()

/home/gerben-koopman/miniconda3/envs/nesymres/lib/python3.8/site-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../weights/100M.ckpt`


In [8]:
from functools import partial
from nesymres.architectures.beam_search import generate_multiple_beams

# Define multiple beam search configurations for ablation study
beam_configs = [
    {"beam_size": 5, "length_penalty": 1.0, "max_len": 100},
    {"beam_size": 10, "length_penalty": 0.8, "max_len": 150},
    {"beam_size": 3, "length_penalty": 1.2, "max_len": 80},
]

fitfunc = partial(model.fitfunc, cfg_params=params_fit, beam_configs=beam_configs)

In [9]:
# Create points from an equation
number_of_points = 500
n_variables = 2

#To get best results make sure that your support inside the max and mix support
max_supp = cfg.dataset_train.fun_support["max"] 
min_supp = cfg.dataset_train.fun_support["min"]
X = torch.rand(number_of_points,len(list(eq_setting["total_variables"])))*(max_supp-min_supp)+min_supp
X[:,n_variables:] = 0
target_eq = "x_1*sin(x_1)+cos(x_2)" #Use x_1,x_2 and x_3 as independent variables
X_dict = {x:X[:,idx].cpu() for idx, x in enumerate(eq_setting["total_variables"])} 
y = lambdify(",".join(eq_setting["total_variables"]), target_eq)(**X_dict)

In [10]:
print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  torch.Size([500, 3])
y shape:  torch.Size([500])


In [11]:
output = fitfunc(X,y) 

/home/gerben-koopman/studie/nesymres/src/nesymres/architectures/model.py:207: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X,device=self.device).unsqueeze(0)
/home/gerben-koopman/studie/nesymres/src/nesymres/architectures/model.py:211: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y,device=self.device).unsqueeze(0)


Memory footprint of the encoder: 0.0001024GB 



/home/gerben-koopman/miniconda3/envs/nesymres/lib/python3.8/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Loss constructed, starting new BFGS optmization...
Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Loss constructed, starting new BFGS optmization...
Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Loss constructed, starting new BFGS optmization...
Loss constructed, starting new BFGS optmization...
Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Loss constructed, starting new BFGS optmization...
Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Loss constructed, starting new BFGS optmization...
Constructi

In [12]:
output

[{'config': {'beam_size': 5, 'length_penalty': 1.0, 'max_len': 100},
  'output': {'all_bfgs_preds': ['(((x_1)*(sin(x_1)))+(cos(x_2)))',
    'x_1*sin(x_1) + 0.999999996114992*cos(x_2)',
    '(((x_1)*(sin(x_1)))+(sin(cos(x_2))))',
    'x_1*cos(x_1 + 4.712388990822) + cos(x_2)',
    'x_1*sin(x_1) + cos(x_2 - 1.4834446309234e-8)'],
   'all_bfgs_loss': [0.0, 0.0, 0.008284129, 2.4116709e-14, 0.0],
   'best_bfgs_preds': ['(((x_1)*(sin(x_1)))+(cos(x_2)))'],
   'best_bfgs_loss': [0.0]}},
 {'config': {'beam_size': 10, 'length_penalty': 0.8, 'max_len': 150},
  'output': {'all_bfgs_preds': ['(((x_1)*(sin(x_1)))+(cos(x_2)))',
    'x_1*sin(x_1) + 0.999999975725089*cos(x_2)',
    '(((x_1)*(sin(x_1)))+(sin(cos(x_2))))',
    'x_1*cos(x_1 + 4.71238894503717) + cos(x_2)',
    'x_1*sin(x_1) + cos(x_2 + 2.94125416714029e-9)',
    '(x_1 - 1.43250354980862e-9)*sin(x_1) + cos(x_2)',
    'x_1*sin(x_1) + cos(0.99999999046535*x_2)',
    '1.00000001471522*x_1*sin(x_1) + cos(x_2)',
    'x_1*sin(x_1) + sin(x_2 - 4.